# <b>House Prices - Advanced Regression Techniques

# <b>Step 0. 라이브러리 import 및 데이터 불러오기
### **가. 라이브러리 import**
                                                                                                                      by kjulous77

In [54]:
import pandas as pd
import numpy as np

### **나.  학습데이터 불러오기**

https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview

In [55]:
df = pd.read_csv('https://raw.githubusercontent.com/DX5-Study/House/main/train.csv', delimiter=',')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# <b>Step 1. 데이터 전처리
---

## <b>1. 중복 데이터 제거


In [56]:
# 데이터 프레임의 info를 확인합니다.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [57]:
# 중복 데이터 제거를 제거합니다.
df.drop_duplicates(inplace=True)

In [58]:
# 데이터 프레임의 info를 확인합니다.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

---

## <b>2. 결측치 처리

In [59]:
def fillknn(df):
    from sklearn.impute import KNNImputer
    imputer = KNNImputer(n_neighbors=5)
    knn_df=imputer.fit_transform(df)
    knn_df=pd.DataFrame(knn_df, columns=df.columns)
    return knn_df

In [60]:
# 결측치 있는 항만 확인
nas =pd.DataFrame(df.isna().sum())
nas.loc[nas[0]!=0]

,0
LotFrontage,259
Alley,1369
MasVnrType,8
MasVnrArea,8
BsmtQual,37
BsmtCond,37
BsmtExposure,38
BsmtFinType1,37
BsmtFinType2,38
Electrical,1


In [61]:
df['Fence'].value_counts()

MnPrv    157
GdPrv     59
GdWo      54
MnWw      11
Name: Fence, dtype: int64

In [62]:
df['MiscFeature'].value_counts()

Shed    49
Gar2     2
Othr     2
TenC     1
Name: MiscFeature, dtype: int64

In [63]:
df['Alley'].value_counts()

Grvl    50
Pave    41
Name: Alley, dtype: int64

In [64]:
# 해당 feature 들은 전체 행 개수에 비해 결측치 개수가 과다함
del_col = ['Fence', 'MiscFeature', 'Alley','Id']
df.drop(del_col, axis=1, inplace=True)

In [65]:
nas =pd.DataFrame(df.isna().sum())
nas.loc[nas[0]!=0]

,0
LotFrontage,259
MasVnrType,8
MasVnrArea,8
BsmtQual,37
BsmtCond,37
BsmtExposure,38
BsmtFinType1,37
BsmtFinType2,38
Electrical,1
FireplaceQu,690


In [66]:
df.loc[df['PoolArea']==0, 'PoolQC'].sum()

0

In [67]:
# PoolArea가 0 인 경우 PoolQC가 결측치 이므로 none으로 처리
df.loc[df['PoolArea']==0, 'PoolQC']='none'

In [68]:
# MasVnrType 이 결측치일 떄, MasVnrArea 도 결측치를 가진다.
df['MasVnrType'].value_counts()

None       864
BrkFace    445
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64

In [69]:
# 해당항들은 결측치가 매우 적으므로 제거
df.dropna(subset=['Electrical', 'MasVnrType'], inplace=True)

In [70]:
nas =pd.DataFrame(df.isna().sum())
nas.loc[nas[0]!=0]

,0
LotFrontage,257
BsmtQual,37
BsmtCond,37
BsmtExposure,38
BsmtFinType1,37
BsmtFinType2,38
FireplaceQu,685
GarageType,81
GarageYrBlt,81
GarageFinish,81


In [71]:
df['BsmtFinType1'].unique()

array(['GLQ', 'ALQ', 'Unf', 'Rec', 'BLQ', nan, 'LwQ'], dtype=object)

In [72]:
df['BsmtFinType2'].unique()

array(['Unf', 'BLQ', nan, 'ALQ', 'Rec', 'LwQ', 'GLQ'], dtype=object)

In [73]:
df['BsmtCond'].unique()

array(['TA', 'Gd', nan, 'Fa', 'Po'], dtype=object)

In [74]:
df[['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']].loc[df['BsmtExposure'].isna()]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2
17,NaN,NaN,NaN,NaN,NaN
39,NaN,NaN,NaN,NaN,NaN
90,NaN,NaN,NaN,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN
156,NaN,NaN,NaN,NaN,NaN
182,NaN,NaN,NaN,NaN,NaN
259,NaN,NaN,NaN,NaN,NaN
342,NaN,NaN,NaN,NaN,NaN
362,NaN,NaN,NaN,NaN,NaN
371,NaN,NaN,NaN,NaN,NaN


In [75]:
# 한개 항을 제외하면 모두 같이 결측치를 가진다. 데이터 설명에 없는 경우가 표기 되어 있으므로, 결측치가 없는 경우로 가정
cols1 = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
df[cols1] =df[cols1].fillna('none')

In [76]:
nas =pd.DataFrame(df.isna().sum())
nas.loc[nas[0]!=0]

,0
LotFrontage,257
FireplaceQu,685
GarageType,81
GarageYrBlt,81
GarageFinish,81
GarageQual,81
GarageCond,81


In [77]:
df['GarageCond'].unique()

array(['TA', 'Fa', nan, 'Gd', 'Po', 'Ex'], dtype=object)

In [78]:
df[['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond']].loc[df['GarageCond'].isna()]

,GarageType,GarageYrBlt,GarageFinish,GarageQual,GarageCond
39,NaN,NaN,NaN,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN
78,NaN,NaN,NaN,NaN,NaN
88,NaN,NaN,NaN,NaN,NaN
89,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
1349,NaN,NaN,NaN,NaN,NaN
1407,NaN,NaN,NaN,NaN,NaN
1449,NaN,NaN,NaN,NaN,NaN
1450,NaN,NaN,NaN,NaN,NaN


In [79]:
# 한개 항을 제외하면 모두 같이 결측치를 가진다. 데이터 설명에 없는 경우가 표기 되어 있으므로, 결측치가 없는 경우로 가정#
cols2 = ['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond']
df[cols2] =df[cols2].fillna('none')

In [80]:
nas =pd.DataFrame(df.isna().sum())
nas.loc[nas[0]!=0]

,0
LotFrontage,257
FireplaceQu,685


In [81]:
df['LotFrontage'].unique()

array([ 65.,  80.,  68.,  60.,  84.,  85.,  75.,  nan,  51.,  50.,  70.,
        91.,  72.,  66., 101.,  57.,  44., 110.,  98.,  47., 108., 112.,
        74., 115.,  61.,  48.,  33.,  52., 100.,  24.,  89.,  63.,  76.,
        81.,  95.,  69.,  21.,  32.,  78., 121., 122.,  40., 105.,  73.,
        77.,  64.,  94.,  34.,  90.,  55.,  88.,  82.,  71., 120., 107.,
        92., 134.,  62.,  86., 141.,  97.,  54.,  41.,  79., 174.,  99.,
        67.,  83.,  43., 103.,  93.,  30., 129., 140.,  35.,  37., 118.,
        87., 116., 150., 111.,  49.,  96.,  59.,  36.,  56., 102.,  58.,
        38., 109., 130.,  53., 137.,  45., 106., 104.,  42.,  39., 144.,
       114., 128., 149., 313., 168., 182., 138., 160., 152., 124., 153.,
        46.])

In [82]:
df['FireplaceQu'].unique()

array([nan, 'TA', 'Gd', 'Fa', 'Ex', 'Po'], dtype=object)

In [83]:
# 데이터 설명에 없는 경우가 표기 되어 있으므로, 결측치가 없는 경우로 가정
df['FireplaceQu'] = df['FireplaceQu'].fillna('none')

In [84]:
nas =pd.DataFrame(df.isna().sum())
nas.loc[nas[0]!=0]

,0
LotFrontage,257


In [85]:
df['GarageCond'].value_counts()

TA      1317
none      81
Fa        35
Gd         9
Po         7
Ex         2
Name: GarageCond, dtype: int64

In [86]:
# 평가를 점수로 라벨링 함
cols3 =['ExterQual', 'ExterCond', 'HeatingQC', 'KitchenQual']
cols4 = ['BsmtQual', 'BsmtCond', 'FireplaceQu', 'GarageQual', 'GarageCond']
df[cols3] =df[cols3].replace({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1})
df[cols4] = df[cols4].replace({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'none':0})
df['PoolQC'] = df['PoolQC'].replace({'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'none':0})
df['CentralAir'] = df['CentralAir'].replace({'Y':1, 'N':0})

In [87]:
df['Electrical'].unique()

array(['SBrkr', 'FuseF', 'FuseA', 'FuseP', 'Mix'], dtype=object)

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1451 entries, 0 to 1459
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1451 non-null   int64  
 1   MSZoning       1451 non-null   object 
 2   LotFrontage    1194 non-null   float64
 3   LotArea        1451 non-null   int64  
 4   Street         1451 non-null   object 
 5   LotShape       1451 non-null   object 
 6   LandContour    1451 non-null   object 
 7   Utilities      1451 non-null   object 
 8   LotConfig      1451 non-null   object 
 9   LandSlope      1451 non-null   object 
 10  Neighborhood   1451 non-null   object 
 11  Condition1     1451 non-null   object 
 12  Condition2     1451 non-null   object 
 13  BldgType       1451 non-null   object 
 14  HouseStyle     1451 non-null   object 
 15  OverallQual    1451 non-null   int64  
 16  OverallCond    1451 non-null   int64  
 17  YearBuilt      1451 non-null   int64  
 18  YearRemo

In [89]:
# 라벨링한 데이터 이외에 분류형 feature
dumm_cols = ['MSZoning', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
             'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',  'Exterior2nd',
             'MasVnrType', 'Foundation', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'Electrical', 'Functional',
             'GarageType', 'GarageYrBlt', 'GarageFinish', 'PavedDrive', 'SaleType', 'SaleCondition', 'YearBuilt', 'YearRemodAdd',
             'GarageYrBlt', 'YrSold', 'MSZoning','Street']

In [90]:
df.drop(dumm_cols, axis=1, inplace=True)

In [91]:
df= fillknn(df)

In [92]:
df.drop_duplicates(inplace=True)

## <b>3. 모델링

#### <b>3.1 모델 설정

In [40]:
from pycaret.regression import *

In [41]:
exp = setup(data = df,  
            target = 'SalePrice',
            normalize = True, 
            normalize_method = 'minmax',
            transformation = False, 
            fold = 5, 
            fold_shuffle=True,        
            silent = True,  
            use_gpu = True)

,Description,Value
0,session_id,2756
1,Target,SalePrice
2,Original Data,"(1451, 44)"
3,Missing Values,False
4,Numeric Features,42
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(1015, 43)"


In [42]:
data_transform = get_config('X') # train 데이터
data_transform.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,MoSold,CentralAir_0.0
0,0.235294,0.150685,0.042649,0.666667,0.500,0.12250,0.666667,0.5,0.8,0.75,...,0.000000,0.111517,0.000000,0.0,0.0,0.0,0.0,0.0,0.090909,0.0
1,0.000000,0.202055,0.049697,0.555556,0.875,0.00000,0.333333,0.5,0.8,0.75,...,0.347725,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.363636,0.0
2,0.235294,0.160959,0.059809,0.666667,0.500,0.10125,0.666667,0.5,0.8,0.75,...,0.000000,0.076782,0.000000,0.0,0.0,0.0,0.0,0.0,0.727273,0.0
3,0.294118,0.133562,0.049391,0.666667,0.500,0.00000,0.333333,0.5,0.6,1.00,...,0.000000,0.063985,0.492754,0.0,0.0,0.0,0.0,0.0,0.090909,0.0
4,0.235294,0.215753,0.078256,0.777778,0.500,0.21875,0.666667,0.5,0.8,0.75,...,0.224037,0.153565,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,0.0


#### <b>3-2. 모델 생성

---

In [43]:
best3 = compare_models( sort='RMSE', n_select = 3, budget_time = 0.5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,17028.8826,965538181.4119,30201.6508,0.8528,0.1402,0.0983,2.4760
rf,Random Forest Regressor,18044.7925,1086684439.5139,31861.2824,0.8330,0.1515,0.1071,0.7300
et,Extra Trees Regressor,18507.7337,1089225340.5999,32231.1670,0.8322,0.1525,0.1083,0.7120
lightgbm,Light Gradient Boosting Machine,18943.7973,1119917355.7753,32642.1151,0.8277,0.1513,0.1088,0.1420
gbr,Gradient Boosting Regressor,18049.8056,1206576969.4339,32949.5850,0.8154,0.1504,0.1064,0.2800
huber,Huber Regressor,20884.2092,1456303646.9639,37360.0152,0.7792,0.1652,0.1179,0.0760
ada,AdaBoost Regressor,24340.0652,1533565027.6120,38593.6641,0.7655,0.2025,0.1576,0.1940
knn,K Neighbors Regressor,24231.9332,1673923302.4000,40526.7172,0.7442,0.1889,0.1362,0.5660
ridge,Ridge Regression,22476.7020,1824260723.2000,40938.8688,0.7224,0.1911,0.1316,0.0060
br,Bayesian Ridge,22665.0186,1959450348.4765,42103.9236,0.7019,0.2029,0.1331,0.0180


In [44]:
best_top3 = [tune_model(i, optimize = 'RMSE', n_iter=20) for i in best3]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,19755.1068,859887899.1367,29323.8452,0.8586,0.1560,0.1168
1,20043.3972,996452244.4029,31566.6318,0.8791,0.1628,0.1197
2,18143.4360,786212725.9031,28039.4851,0.8522,0.1411,0.1024
3,18078.8587,995062496.2783,31544.6112,0.8535,0.1431,0.1015
4,19939.0653,1813845760.5110,42589.2681,0.7217,0.1738,0.1215
Mean,19191.9728,1090292225.2464,32612.7683,0.8330,0.1554,0.1124
Std,887.5408,370662881.0897,5167.1626,0.0565,0.0123,0.0087


In [45]:
blend_3_soft = blend_models(estimator_list = best3, fold=5, optimize='RMSE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,17247.5216,669605322.3065,25876.7332,0.8899,0.1388,0.1019
1,17156.4982,712139971.5999,26685.9508,0.9136,0.1487,0.1049
2,15533.3338,593781836.2916,24367.6391,0.8884,0.1282,0.0902
3,16565.4830,865551997.0438,29420.2651,0.8725,0.1296,0.0918
4,18439.2861,2078087928.0880,45586.0497,0.6812,0.1686,0.1100
Mean,16988.4245,983833411.0659,30387.3276,0.8491,0.1428,0.0997
Std,948.3193,554270744.0817,7774.5568,0.0850,0.0149,0.0076


In [46]:
stacker_3 = stack_models(estimator_list = best3[1:], meta_model = best3[0]) 

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,15860.0281,595264678.4128,24398.0466,0.9021,0.1331,0.0940
1,17887.2368,717961754.0784,26794.8083,0.9129,0.1514,0.1089
2,15511.3555,578269417.1967,24047.2330,0.8913,0.1282,0.0893
3,17542.0686,1059766956.5342,32554.0621,0.8439,0.1326,0.0941
4,19741.7770,2355303458.6736,48531.4687,0.6386,0.1746,0.1155
Mean,17308.4932,1061313252.9792,31265.1238,0.8378,0.1440,0.1004
Std,1525.6557,669751583.2614,9154.5229,0.1023,0.0173,0.0100


## <b>4. 예측

In [47]:
test = pd.read_csv('https://raw.githubusercontent.com/DX5-Study/House/main/test.csv')
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [48]:
from matplotlib.pyplot import axis


def set_data(df):
    del_col = ['Fence', 'MiscFeature', 'Alley','Id']
    data_to1 = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
    data_to2 =['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond']
    data_to3 = ['ExterQual', 'ExterCond', 'HeatingQC', 'KitchenQual']
    data_to4 = ['BsmtQual', 'BsmtCond', 'FireplaceQu', 'GarageQual', 'GarageCond']
    df.drop(del_col, axis=1, inplace=True)
    df.loc[test['PoolArea']==0, 'PoolQC']='none'
    df[data_to1] =df[data_to1].fillna('none')
    df[data_to2] =df[data_to2].fillna('none')
    df['FireplaceQu'] = df['FireplaceQu'].fillna('none')
    df[data_to3] =df[data_to3].replace({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1})
    df[data_to4] = df[data_to4].replace({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'none':0})
    df['PoolQC'] = df['PoolQC'].replace({'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'none':0})
    df['CentralAir'] = df['CentralAir'].replace({'Y':1, 'N':0})
    df.drop(dumm_cols, axis=1, inplace=True)
    return df

In [49]:
# 전처리
test = set_data(test)
# knn imputer
test = fillknn(test)

In [93]:
# stacker_3, blend_3_soft
final_model = finalize_model(stacker_3)

In [94]:
pred = predict_model(final_model, data = test)

In [95]:
submission =pd.read_csv('https://raw.githubusercontent.com/DX5-Study/House/main/sample_submission.csv')
submission['SalePrice'] = pred[['Label']]
submission.head()

,Id,SalePrice
0,1461,125026.836069
1,1462,170853.646231
2,1463,175691.534400
3,1464,176485.650993
4,1465,182619.594488


In [96]:
#  캐글에서 ERROR: Column '' was not expected (Line 1, Column 1)  때문에 index=False 추가
submission.to_csv('submission.csv',  index=False)

![](https://file%2B.vscode-resource.vscode-cdn.net/c%3A/Users/Hyun/Desktop/%EC%83%88%20%ED%8F%B4%EB%8D%94/kwon/score.png?version%3D1664806354989)